# Category 테이블 Mapping 및 Table join

성별 추론 실습 전 필요한 Mapping 테이블을 생성하여 원본 테이블과 Join하는 예제입니다.

## 1) 공통 셋업 & 유틸

In [ ]:
import time, typing as T, re, json
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import Counter

import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

BASE_URL = "http://61.109.236.101"

OUT_DIR = Path("datasets") / "gender"; OUT_DIR.mkdir(exist_ok=True)
PRODUCT_MAP_PARQUET = OUT_DIR / "product_category_map.parquet"
PRODUCT_MAP_CSV     = OUT_DIR / "product_category_map.csv"
KEYWORD_MAP_CSV     = OUT_DIR / "keyword_to_category_map.csv"
KEYWORD_MAP_PARQUET = OUT_DIR / "keyword_to_category_map.parquet"
DATASET_PATH = "datasets/processed_user_behavior.sorted.csv"
JOIN_OUT_CSV = OUT_DIR / "processed_user_behavior.joined.csv"
JOIN_OUT_PQ  = OUT_DIR / "processed_user_behavior.joined.parquet"

TIMEOUT = 5
SLEEP   = 0.15
MAX_WORKERS = 8

def make_sess() -> requests.Session:
    s = requests.Session()
    r = Retry(
        total=3,
        backoff_factor=0.4,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD", "OPTIONS"],
    )
    a = HTTPAdapter(max_retries=r, pool_connections=10, pool_maxsize=32)
    s.mount("http://", a); s.mount("https://", a)
    s.headers.update({"User-Agent": "cat-mapper/1.0", "Accept": "text/html,*/*;q=0.8"})
    return s

S = make_sess()

def bs_text(html: str) -> str:
    return BeautifulSoup(html, "html.parser").get_text("\n", strip=True)

def parse_category_from_html(html: str) -> T.Optional[str]:
    text = bs_text(html)
    m = re.search(r"Category\s*:\s*(.+)", text, flags=re.I)
    if m:
        cat = m.group(1)
        for stop in ["Average Rating:", "Reviews:", "Price:", "ID:", "Name:", "[Home]"]:
            if stop in cat:
                cat = cat.split(stop)[0]
        cat = cat.splitlines()[0].strip()
        return cat or None
    for ln in text.splitlines():
        if ln.lower().startswith("category:"):
            return ln.split(":", 1)[1].strip() or None
    return None

def product_category(pid: T.Union[int, str]) -> T.Optional[str]:
    """GET /product?id=<pid> → category 문자열 또는 None"""
    url = f"{BASE_URL}/product"
    for attempt in range(3):
        try:
            time.sleep(SLEEP if attempt else 0)
            r = S.get(url, params={"id": pid}, timeout=TIMEOUT)
            if not r.encoding:
                r.encoding = "utf-8"
            if r.status_code == 404:
                return None
            r.raise_for_status()
            return parse_category_from_html(r.text)
        except Exception:
            if attempt < 2:
                time.sleep(0.5 * (2 ** attempt))  # 0.5s → 1s
                continue
            return None

PROD_RE = re.compile(r"/product\?id=(\d+)", re.I)

def search_ids(q: str, max_products: int = 5) -> list[int]:
    """GET /search?query=q → product id 리스트(최대 max_products)"""
    time.sleep(SLEEP)
    r = S.get(f"{BASE_URL}/search", params={"query": q}, timeout=TIMEOUT)
    if r.status_code != 200:
        return []
    ids: list[int] = []
    soup = BeautifulSoup(r.text, "html.parser")
    for a in soup.find_all("a", href=True):
        m = PROD_RE.search(a["href"])
        if m:
            ids.append(int(m.group(1)))
    if not ids:
        ids = [int(x) for x in PROD_RE.findall(r.text)] 
    ids = list(dict.fromkeys(ids))
    return ids[:max_products]

def clean_cat(x):
    if pd.isna(x):
        return x
    return re.sub(r"</?[^>]+>", "", str(x)).strip()

print("[Done]")

## 2) Product -> Category Mapping 테이블 생성

In [ ]:
product_ids = [str(i) for i in range(101, 125)]

if PRODUCT_MAP_PARQUET.exists():
    existing = pd.read_parquet(PRODUCT_MAP_PARQUET)
    if "category_name" in existing.columns and "category" not in existing.columns:
        existing = existing.rename(columns={"category_name": "category"})
    existing = existing[["product_id", "category"]]
else:
    existing = pd.DataFrame(columns=["product_id", "category"])

have = set(existing["product_id"].astype(str))
todo = [pid for pid in product_ids if pid not in have]
print(f"[plan] total={len(product_ids)} have={len(have)} to_fetch={len(todo)}")

rows = []
if todo:
    def fetch_one(pid: str) -> dict:
        cat = product_category(pid)
        return {"product_id": pid, "category": cat}

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futs = {ex.submit(fetch_one, pid): pid for pid in todo}
        for fut in as_completed(futs):
            rows.append(fut.result())

new_df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=["product_id", "category"])

merged_all = pd.concat([existing, new_df], ignore_index=True)
if not merged_all.empty:
    merged_all["is_valid"] = merged_all["category"].notna().astype(int)
    product_map_df = (
        merged_all.sort_values(["product_id", "is_valid"], ascending=[True, False])
        .drop_duplicates(subset=["product_id"], keep="first")
        .drop(columns=["is_valid"])
        .reset_index(drop=True)
    )
else:
    product_map_df = merged_all

product_map_df.to_parquet(PRODUCT_MAP_PARQUET, index=False)
product_map_df.to_csv(PRODUCT_MAP_CSV, index=False, encoding="utf-8")
print(f"[done] saved:\n  - {PRODUCT_MAP_PARQUET} (rows={len(product_map_df)})\n  - {PRODUCT_MAP_CSV}")

product_map_df.head(24)


## 3) Keyword -> Category Mapping 테이블 생성

In [ ]:
URLENC_RE = re.compile(r"%[0-9A-Fa-f]{2}")
df_src = pd.read_csv(DATASET_PATH)

if "search_keyword" in df_src.columns:
    raw = df_src["search_keyword"].dropna().astype(str).str.strip()
    raw = raw[(raw != "") & (~raw.str.contains(URLENC_RE))]
else:
    raw = pd.Series(dtype=str)

canon = raw.str.lower()
freq = (
    pd.DataFrame({"canon": canon, "orig": raw})
    .groupby(["canon", "orig"]).size()
    .reset_index(name="cnt")
    .sort_values(["canon", "cnt"], ascending=[True, False])
)
variants_map = freq.groupby("canon")["orig"].apply(list).to_dict()
canons = sorted(variants_map.keys())
print("키워드 수:", len(canons))

MAX_PRODUCTS_PER_KEYWORD = 5

def infer_for_canon(canon_kw: str):
    tried = []
    tried += variants_map.get(canon_kw, [])
    for q in [canon_kw.title(), canon_kw, canon_kw.upper()]:
        if q not in tried:
            tried.append(q)

    picked_ids = []
    for q in tried:
        picked_ids = search_ids(q, max_products=MAX_PRODUCTS_PER_KEYWORD)
        if picked_ids:
            break

    counts = Counter(); checked = 0
    for pid in picked_ids:
        cat = product_category(pid)
        if cat:
            counts[cat] += 1
            checked += 1

    if checked == 0:
        return {"keyword": canon_kw, "top_category": None}

    top_cat, _ = counts.most_common(1)[0]
    return {"keyword": canon_kw, "top_category": top_cat}

rows = []
for i, ckw in enumerate(canons, 1):
    rows.append(infer_for_canon(ckw))
    if i % 20 == 0 or i == len(canons):
        print(f"[{i}/{len(canons)}] 진행 중…")

kw_map_df = (
    pd.DataFrame(rows)[["keyword", "top_category"]]
    .sort_values(["top_category", "keyword"], na_position="last")
    .reset_index(drop=True)
)

kw_map_df.to_csv(KEYWORD_MAP_CSV, index=False, encoding="utf-8-sig")
kw_map_df.to_parquet(KEYWORD_MAP_PARQUET, index=False)
print(f"[done] keyword map saved:\n  - {KEYWORD_MAP_CSV}\n  - {KEYWORD_MAP_PARQUET}")

kw_map_df.head(25)

## 4) Mapping 테이블들과 원본 테이블 Join

In [ ]:
df = pd.read_csv(DATASET_PATH)

kw_map = pd.read_csv(KEYWORD_MAP_CSV).rename(columns={"top_category": "kw_category"})
kw_map["keyword"] = kw_map["keyword"].astype(str).str.strip().str.lower()
kw_series = kw_map.drop_duplicates(subset=["keyword"]).set_index("keyword")["kw_category"]

prod_map = pd.read_csv(PRODUCT_MAP_CSV)
if "category" in prod_map.columns:
    prod_map = prod_map.rename(columns={"category": "prod_category"})
elif "category_name" in prod_map.columns:
    prod_map = prod_map.rename(columns={"category_name": "prod_category"})
elif "top_category" in prod_map.columns:
    prod_map = prod_map.rename(columns={"top_category": "prod_category"})
else:
    raise ValueError("product_category_map.csv 에 category/category_name/top_category 중 하나가 필요합니다.")

prod_map["product_id"] = pd.to_numeric(prod_map["product_id"], errors="coerce")
prod_map = prod_map.dropna(subset=["product_id"]).drop_duplicates(subset=["product_id"], keep="first")
prod_series = prod_map.set_index("product_id")["prod_category"]

if "product_id" in df.columns:
    pid_key = pd.to_numeric(df["product_id"], errors="coerce")
    prod_hit = pid_key.map(prod_series)
else:
    prod_hit = pd.Series([pd.NA] * len(df), index=df.index)

if "search_keyword" in df.columns:
    kw_key = df["search_keyword"].astype(str).str.strip().str.lower()
    kw_hit = kw_key.map(kw_series)
else:
    kw_hit = pd.Series([pd.NA] * len(df), index=df.index)

resolved = prod_hit.combine_first(kw_hit)
if "category_name" in df.columns:
    resolved = resolved.combine_first(df["category_name"])
df["resolved_category"] = resolved.map(clean_cat)

cols = df.columns.tolist()
if "product_id" in cols:
    insert_at = cols.index("product_id") + 1
    cols = [c for c in cols if c != "resolved_category"]
    cols = cols[:insert_at] + ["resolved_category"] + cols[insert_at:]
    df = df[cols]

df.to_csv(JOIN_OUT_CSV, index=False, encoding="utf-8-sig")
print(f"[OK] CSV 저장: {JOIN_OUT_CSV} (rows={len(df)})")
try:
    df.to_parquet(JOIN_OUT_PQ, index=False)
    print(f"[OK] Parquet 저장: {JOIN_OUT_PQ}")
except Exception as e:
    print("[WARN] Parquet 저장 실패:", e, "→ pyarrow 또는 fastparquet 설치 필요")
